In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!git clone https://github.com/nanjunye/SDNN_ipynb.git

Cloning into 'SDNN_ipynb'...
remote: Enumerating objects: 932, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 932 (delta 11), reused 16 (delta 5), pack-reused 906
Receiving objects: 100% (932/932), 13.86 MiB | 37.86 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import sys
sys.path.append('/content/SDNN_ipynb/src')

In [3]:
from SDNN_cuda import SDNN
from Classifier import Classifier
import numpy as np
import os
from math import floor
import time

DeprecationError: ignored

In [4]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

--2023-03-14 06:28:43--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  77.5MB/s    in 3.2s    

2023-03-14 06:28:46 (73.6 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [16]:
!unzip -qq 'tiny-imagenet-200.zip' 

unzip:  cannot find or open tiny-imagenet-200.zip, tiny-imagenet-200.zip.zip or tiny-imagenet-200.zip.ZIP.


In [ ]:
DATA_DIR = 'drive/MyDrive/Colab Notebooks/SDNN_ipynb/datasets'

TRAIN_DIR = os.path.join(DATA_DIR, 'train') 
VALID_DIR = os.path.join(DATA_DIR, 'val')
VALID_DIR = os.path.join(DATA_DIR, 'test')

In [ ]:
#Settings
start = time.time()
# Flags
learn_SDNN = True  # This flag toggles between Learning STDP and classify features
# or just classify by loading pretrained weights for the face/motor dataset
if learn_SDNN:
    set_weights = False  # Loads the weights from a path (path_set_weigths) and prevents any SDNN learning
    save_weights = True  # Saves the weights in a path (path_save_weigths)
    save_features = True  # Saves the features and labels in the specified path (path_features)
else:
    set_weights = True  # Loads the weights from a path (path_set_weigths) and prevents any SDNN learning
    save_weights = False  # Saves the weights in a path (path_save_weigths)
    save_features = False  # Saves the features and labels in the specified path (path_features)


In [ ]:
# ------------------------------- Learn, Train and Test paths-------------------------------#
ipynb_path = '/content/drive/MyDrive/Colab Notebooks/SDNN_ipynb'
# Image sets directories
spike_times_learn = [ipynb_path + '/datasets/LearningSet/Face/', ipynb_path + '/datasets/LearningSet/Motor/']
spike_times_train = [ipynb_path + '/datasets/TrainingSet/Face/', ipynb_path + '/datasets/TrainingSet/Motor/']
spike_times_test = [ipynb_path + '/datasets/TestingSet/Face/', ipynb_path + '/datasets/TestingSet/Motor/']

# Results directories
path_set_weigths = [ipynb_path + 'results/']
path_save_weigths = [ipynb_path + 'results/']
path_features = [ipynb_path + 'results/']

In [ ]:
# ------------------------------- SDNN -------------------------------#
# SDNN_cuda parameters
DoG_params = {'img_size': (64, 64), 'DoG_size': 7, 'std1': 1., 'std2': 2.}  # img_size is (col size, row size)
total_time = 15
network_params = [{'Type': 'input', 'num_filters': 1, 'pad': (0, 0), 'H_layer': DoG_params['img_size'][1],
                       'W_layer': DoG_params['img_size'][0]},
                      {'Type': 'conv', 'num_filters': 4, 'filter_size': 5, 'th': 10.},
                      {'Type': 'pool', 'num_filters': 4, 'filter_size': 7, 'th': 0., 'stride': 6},
                      {'Type': 'conv', 'num_filters': 20, 'filter_size': 17, 'th': 60.},
                      {'Type': 'pool', 'num_filters': 20, 'filter_size': 5, 'th': 0., 'stride': 5},
                      {'Type': 'conv', 'num_filters': 20, 'filter_size': 5, 'th': 2.}]

weight_params = {'mean': 0.8, 'std': 0.01}

max_learn_iter = [0, 3000, 0, 5000, 0, 6000, 0]
stdp_params = {'max_learn_iter': max_learn_iter,
                   'stdp_per_layer': [0, 10, 0, 4, 0, 2],
                   'max_iter': sum(max_learn_iter),
                   'a_minus': np.array([0, .003, 0, .0003, 0, .0003], dtype=np.float32),
                   'a_plus': np.array([0, .004, 0, .0004, 0, .0004], dtype=np.float32),
                   'offset_STDP': [0, floor(network_params[1]['filter_size']),
                                   0,
                                   floor(network_params[3]['filter_size']/8),
                                   0,
                                   floor(network_params[5]['filter_size'])]}

In [ ]:
# Create network
first_net = SDNN(network_params, weight_params, stdp_params, total_time,
                     DoG_params=DoG_params, spike_times_learn=spike_times_learn,
                     spike_times_train=spike_times_train, spike_times_test=spike_times_test, device='GPU')


In [ ]:
# Set the weights or learn STDP
if set_weights:
    weight_path_list = ['/content/drive/MyDrive/Colab Notebooks/SDNN_ipynb/results'  + 'weight_' + str(i) + '.npy' for i in range(len(network_params) - 1)]
    first_net.set_weights(weight_path_list)
else:
    first_net.train_SDNN()


-----------------------------------------------------------
-------------------- STARTING LEARNING---------------------
-----------------------------------------------------------
----------------- Learning Progress  0/14000 (0.0)%----------------------


/content/drive/MyDrive/Colab Notebooks/SDNN_ipynb/src/DoG_filt_cuda.py:64: RuntimeWarning: divide by zero encountered in true_divide
  I = np.argsort(1 / img_out.flatten())  # Get indices of sorted latencies
/content/drive/MyDrive/Colab Notebooks/SDNN_ipynb/src/DoG_filt_cuda.py:65: RuntimeWarning: divide by zero encountered in true_divide
  lat = np.sort(1 / img_out.flatten())  # Get sorted latencies
/usr/local/lib/python3.9/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


----------------- Learning Progress  1/14000 (0.007142857142857143)%----------------------
----------------- Learning Progress  2/14000 (0.014285714285714285)%----------------------
----------------- Learning Progress  3/14000 (0.02142857142857143)%----------------------
----------------- Learning Progress  4/14000 (0.02857142857142857)%----------------------
----------------- Learning Progress  5/14000 (0.03571428571428571)%----------------------
----------------- Learning Progress  6/14000 (0.04285714285714286)%----------------------
----------------- Learning Progress  7/14000 (0.05)%----------------------
----------------- Learning Progress  8/14000 (0.05714285714285714)%----------------------
----------------- Learning Progress  9/14000 (0.06428571428571428)%----------------------
----------------- Learning Progress  10/14000 (0.07142857142857142)%----------------------
----------------- Learning Progress  11/14000 (0.07857142857142857)%----------------------
----------------- Lea

/usr/local/lib/python3.9/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 9 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.9/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 18 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Streaming output truncated to the last 5000 lines.
----------------- Learning Progress  3003/14000 (21.45)%----------------------
----------------- Learning Progress  3004/14000 (21.457142857142856)%----------------------
----------------- Learning Progress  3005/14000 (21.464285714285715)%----------------------
----------------- Learning Progress  3006/14000 (21.47142857142857)%----------------------
----------------- Learning Progress  3007/14000 (21.478571428571428)%----------------------
----------------- Learning Progress  3008/14000 (21.485714285714284)%----------------------
----------------- Learning Progress  3009/14000 (21.492857142857144)%----------------------
----------------- Learning Progress  3010/14000 (21.5)%----------------------
----------------- Learning Progress  3011/14000 (21.507142857142856)%----------------------
----------------- Learning Progress  3012/14000 (21.514285714285716)%----------------------
----------------- Learning Progress  3013/14000 (21.52142

/usr/local/lib/python3.9/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Streaming output truncated to the last 5000 lines.
----------------- Learning Progress  9004/14000 (64.31428571428572)%----------------------
----------------- Learning Progress  9005/14000 (64.32142857142857)%----------------------
----------------- Learning Progress  9006/14000 (64.32857142857142)%----------------------
----------------- Learning Progress  9007/14000 (64.33571428571429)%----------------------
----------------- Learning Progress  9008/14000 (64.34285714285714)%----------------------
----------------- Learning Progress  9009/14000 (64.35)%----------------------
----------------- Learning Progress  9010/14000 (64.35714285714286)%----------------------
----------------- Learning Progress  9011/14000 (64.36428571428571)%----------------------
----------------- Learning Progress  9012/14000 (64.37142857142857)%----------------------
----------------- Learning Progress  9013/14000 (64.37857142857143)%----------------------
----------------- Learning Progress  9014/14000 (64

In [ ]:
# Save the weights
ipynb_path2 = '/content/drive/MyDrive/Colab Notebooks/SDNN_ipynb/results'
if save_weights:
    weights = first_net.get_weights()
    for i in range(len(weights)):
        pathsvwei = ipynb_path2 +'/weight_'+f"{i}"
        np.save(pathsvwei, weights[i])


In [ ]:
# Get features
X_train, y_train = first_net.train_features()
X_test, y_test = first_net.test_features()


-----------------------------------------------------------
----------- EXTRACTING TRAINING FEATURES ------------------
-----------------------------------------------------------
------------ Train features Extraction Progress  0/400 (0.0)%----------------
0.29726985700017394
------------ Train features Extraction Progress  1/400 (0.25)%----------------
0.281288675000269
------------ Train features Extraction Progress  2/400 (0.5)%----------------
0.3821584620000067
------------ Train features Extraction Progress  3/400 (0.75)%----------------
0.26975088200015307
------------ Train features Extraction Progress  4/400 (1.0)%----------------
0.2803756470002554
------------ Train features Extraction Progress  5/400 (1.25)%----------------
0.29356561699978556
------------ Train features Extraction Progress  6/400 (1.5)%----------------
0.28565879100005986
------------ Train features Extraction Progress  7/400 (1.75)%----------------
0.2815365239998755
------------ Train features Extractio

In [ ]:
# Save X_train and X_test
if save_features:
    np.save(ipynb_path + '/X_train', X_train)
    np.save(ipynb_path + '/y_train', y_train)
    np.save(ipynb_path + '/X_test', X_test)
    np.save(ipynb_path + '/y_test', y_test)


In [ ]:
# ------------------------------- Classify -------------------------------#
classifier_params = {'C': 1.0, 'gamma': 'auto'}
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)
X_train -= train_mean
X_test -= train_mean
X_train /= (train_std + 1e-5)
X_test /= (train_std + 1e-5)
svm = Classifier(X_train, y_train, X_test, y_test, classifier_params, classifier_type='SVM')
train_score, test_score = svm.run_classiffier()
print('Train Score: ' + str(train_score))
print('Test Score: ' + str(test_score))
print('DONE')

end = time.time()
print(end-start)

kernel was not specified
probability_flag was not specified
Train Score: 0.9375
Test Score: 0.9065656565656566
DONE
4224.832654953003
